
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [14]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race == 'b'].call)

157.0

In [4]:
# number of callbacks for white-sounding names
sum(data[data.race == 'w'].call)

235.0

In [5]:
data.columns

Index([u'id', u'ad', u'education', u'ofjobs', u'yearsexp', u'honors',
       u'volunteer', u'military', u'empholes', u'occupspecific', u'occupbroad',
       u'workinschool', u'email', u'computerskills', u'specialskills',
       u'firstname', u'sex', u'race', u'h', u'l', u'call', u'city', u'kind',
       u'adid', u'fracblack', u'fracwhite', u'lmedhhinc', u'fracdropout',
       u'fraccolp', u'linc', u'col', u'expminreq', u'schoolreq', u'eoe',
       u'parent_sales', u'parent_emp', u'branch_sales', u'branch_emp', u'fed',
       u'fracblack_empzip', u'fracwhite_empzip', u'lmedhhinc_empzip',
       u'fracdropout_empzip', u'fraccolp_empzip', u'linc_empzip', u'manager',
       u'supervisor', u'secretary', u'offsupport', u'salesrep', u'retailsales',
       u'req', u'expreq', u'comreq', u'educreq', u'compreq', u'orgreq',
       u'manuf', u'transcom', u'bankreal', u'trade', u'busservice',
       u'othservice', u'missind', u'ownership'],
      dtype='object')

In [6]:
data.firstname.value_counts().head()

Tamika     256
Anne       242
Allison    232
Latonya    230
Emily      227
Name: firstname, dtype: int64

In [7]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1,0,1,0,0,0,0,0,0,
1,b,1,3,3,6,0,1,1,0,316,...,1,0,1,0,0,0,0,0,0,
2,b,1,4,1,6,0,0,0,0,19,...,1,0,1,0,0,0,0,0,0,
3,b,1,3,4,6,0,1,0,1,313,...,1,0,1,0,0,0,0,0,0,
4,b,1,3,3,22,0,0,0,0,313,...,1,1,0,0,0,0,0,1,0,Nonprofit


In [22]:
import statsmodels.api as sm

dummies = pd.get_dummies(data['race'], prefix='race')

data = pd.concat([data, dummies], axis=1)
train_cols = ['race_b', 'yearsexp', 'ofjobs', 'education']

log_reg = sm.Logit(data['call'], data[train_cols], missing='drop')
results = log_reg.fit()
results.summary()


Optimization terminated successfully.
         Current function value: 0.284390
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   call   No. Observations:                 4870
Model:                          Logit   Df Residuals:                     4866
Method:                           MLE   Df Model:                            3
Date:                Mon, 08 Feb 2016   Pseudo R-squ.:                -0.01578
Time:                        19:04:00   Log-Likelihood:                -1385.0
converged:                       True   LL-Null:                       -1363.5
                                        LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
race_b        -0.6251      0.103     -6.044      0.000        -0.828    -0.422
yearsexp       0.0290      0.010      2.899      0.004         0.009     0.049
ofjobs        -0.1985      0.044     -4.548      0.000        -0.284    -0.113
education     -0.4566      0.041    -11.055      0.000        -0.538    -0.376
==============================================================================
"""

Controlling for years of experience, number of jobs, and education, we see that being black has a negative effect on your prospect of being called. In fact, we can get the odds ratio:

In [37]:
from math import exp
print "Confidence Interval for odds ratio ({0:.2}, {1:.2})".format(
    exp(results.conf_int().loc['race_b'][0]),
    exp(results.conf_int().loc['race_b'][1])
    )


Confidence Interval for odds ratio (0.44, 0.66)


Thus we can see that being black makes you roughly hafl as likely to get a callback, and the result is statistically significant.